# Final Project Phase 2 Summary

For best use of this notebook, open inside the cloned project github repository.

[cryptocarbons](https://github.com/tynanpurdy/cryptocarbons)

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset #1 Source: [Global Power Plant Database](https://datasets.wri.org/dataset/globalpowerplantdatabase) `CSV` and [Global Power Plant Emissions Database](http://meicmodel.org/?page_id=91&lang=en) `XLSX`
*   Downloaded Dataset #2 Source: [BlockChain.com](https://www.blockchain.com/charts/hash-rate) `CSV`
*   Web Collection #1 Source: [WhatToMine GPUs](https://whattomine.com/gpus) `WEBSCRAPE`
*   Web Collection #2 Source: [WhatToMine Coins](https://whattomine.com/coins.json) `API REQUEST`



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import json

In [6]:
def global_power():
  # IMPORT FILES
    generation_path = "data\global-power-plants\global_power_plant_database.csv"
    emission_path = "data\global-power-plants\global_power_emissions_database.xlsx"
  
  # power generation csv
    with open(generation_path, encoding='utf8') as fin:
        ppg = pd.read_csv(fin, low_memory=False)
  
  # power plant emissions xlsx
    ppe = pd.read_excel(emission_path, sheet_name='GPED_v1.0_Plant Level', skiprows=0, header=1)

  # removing unwanted data
    unwanted_columns = ['latitude',
                      'longitude',
                      'other_fuel1',
                      'other_fuel2',
                      'other_fuel3',
                      'commissioning_year',
                      'gppd_idnr',
                      'owner',
                      'source',
                      'url',
                      'geolocation_source',
                      'wepp_id',
                      'year_of_capacity_data',
                      'generation_data_source',
                      'generation_gwh_2018',
                      'generation_gwh_2019',
                      'estimated_generation_note_2013',
                      'estimated_generation_note_2014',
                      'estimated_generation_note_2015',
                      'estimated_generation_note_2016',
                      'estimated_generation_note_2017']
    ppg.drop(unwanted_columns, axis=1, inplace=True)

    unwanted_columns = ['No.', 'Number of Units', 'Total Plant Installed Capacity (MW)']
    ppe.drop(unwanted_columns, axis=1, inplace=True)

  # AGGREGATING DATA
    avgs = ['generation_gwh_2013',
          'generation_gwh_2014',
          'generation_gwh_2015',
          'generation_gwh_2016',
          'generation_gwh_2017']
    ppg['AVG_GENERATION'] = ppg[avgs].mean(axis=1)

    avgs = ['estimated_generation_gwh_2013',
          'estimated_generation_gwh_2014',
          'estimated_generation_gwh_2015',
          'estimated_generation_gwh_2016',
          'estimated_generation_gwh_2017']
    ppg['AVG_EST_GENERATION'] = ppg[avgs].mean(axis=1)

  # merge the two average columns into a single column
    ppg['GENERATION_MW'] = ppg.apply(lambda x : np.fmax(x['AVG_GENERATION'], x['AVG_EST_GENERATION']), axis=1)

  # remove unaggregated columns
    unwanted_columns = ['AVG_GENERATION',
                      'AVG_EST_GENERATION',
                      'generation_gwh_2013',
                      'generation_gwh_2014',
                      'generation_gwh_2015',
                      'generation_gwh_2016',
                      'generation_gwh_2017',
                      'estimated_generation_gwh_2013',
                      'estimated_generation_gwh_2014',
                      'estimated_generation_gwh_2015',
                      'estimated_generation_gwh_2016',
                      'estimated_generation_gwh_2017']
    ppg.drop(unwanted_columns, axis=1, inplace=True)

  # merge rows by fuel type
    generation_dist = ppg.groupby('primary_fuel')['GENERATION_MW'].sum().sort_values(ascending=False)
    emission_dist = ppe.groupby('Fuel Types').aggregate({'CO2 Emissions (Mg)':'sum',
                                                       'SO2 Emissions (Mg)':'sum',
                                                       'NOx Emissions (Mg)':'sum',
                                                       'PM2.5 Emissions (Mg)':'sum'})

  # COMBINING TABLES
    generation_dist.index = generation_dist.index.str.upper()
    generation_dist.drop('WAVE AND TIDAL', axis=0, inplace=True)
    gen_other = ['PETCOKE','WASTE','COGENERATION','STORAGE','NUCLEAR']
    generation_dist['OTHER'] = generation_dist[gen_other].sum(axis=0)
    generation_dist.drop(gen_other, axis=0, inplace=True)
    emission_dist = emission_dist.rename(index={'NG':'GAS'})

    power = pd.concat([generation_dist, emission_dist], axis=1)
    power = power.fillna(0)

    return power

############ Function Call ############
global_power()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\global_power_plant_database.csv'

## Web Collection Requirement \#1


In [25]:
def web_scrape(): 

    #creates a dict that connects to a list of all the HashRates of GPUs for easy iteration when creating visuals
    hashlist = []
    hashdict = {}
    url = requests.get('https://whattomine.com/gpus')
    soup = BeautifulSoup(url.text, 'html.parser')
    for hasher in soup.find_all('div',{'class' :'position-relative'}):
        for h in hasher.stripped_strings:
            hashlist.append(h)
            hashlist2=  hashlist[::2]
            hashdict['Hashrate(Millions of Hash Per Sec)'] = [z for z in hashlist2] 
            
    wattlist = []
    wattdict = {}
    for wat in soup.find_all('small',{'class':'text-muted position-absolute'}):
        for w in wat.stripped_strings:
            wattlist.append(w[-4:-1])
    #return wattlist
            wattdict["Watt"] = [z for z in wattlist]
    
    #creates a dict that connects to a list of all the Revenues of GPUs for easy iteration when creating visuals
    revlist = []
    revdict = {}
    for rev in soup.find_all('td',{'class':'text-right table-success font-weight-bold'}):
        for r in rev.stripped_strings:
            revlist.append(r)
            revdict["24Hour Revenue"] = [z for z in revlist]
    
    #creates a dict that connects to a list of all the Names of GPUs
    namelist = []
    namedict = {}
    for name in soup.find_all('td'):
        for n in name.stripped_strings:
            namelist.append(n)
            if '(*)' in namelist:
                namelist.remove('(*)')
            namelist2 = namelist[1:650:16]
            namedict["GPU Model"] = [z for z in namelist2]

    # this dictionary matches GPU model to the hash rate and 24 hour revenue
    fulldict = {}
    for i in range(len(namelist2)):
        fulldict[namelist2[i]] = {"Hashrate(Millions of Hash Per Sec)":hashlist2[i],"24Hour Revenue": revlist[i]}
    
    #creates a data frame with the columns as GPU name and the index as the description 
    data = []
    data.append(hashlist2)
    data.append(revlist)
    data.append(wattlist)
    df = pd.DataFrame(data, index= ['Hashrate (Millions of Hashes Per Sec)','24 Hour Revenue','Watts'], columns = namelist2).T
    return df 

############ Function Call ############
web_scrape()

,Hashrate (Millions of Hashes Per Sec),24 Hour Revenue,Watts
GeForce RTX 3090,114.00 Mh/s,$6.77,320
Radeon VII,93.00 Mh/s,$5.67,200
GeForce RTX 3080,91.50 Mh/s,$5.50,230
GeForce RTX 3080 Ti,43.00 Mh/s,$4.26,280
Radeon RX 6800,64.00 Mh/s,$3.87,150
Radeon RX 6900 XT,64.00 Mh/s,$3.87,150
Radeon RX 6800 XT,64.00 Mh/s,$3.87,150
GeForce RTX 3060 Ti,58.10 Mh/s,$3.53,130
GeForce RTX 3070,58.10 Mh/s,$3.53,130
GeForce RTX 3070 Ti,35.00 Mh/s,$3.46,230


## Web Collection Requirement \#2

In [8]:
#data with details on specific crypto - to utilize when visualizing 
def coin_info():
    url = requests.get('https://whattomine.com/coins.json')
    j = url.json()
    df = pd.DataFrame(j['coins'])
    coins = pd.Series(j['coins'].keys)
    df.rename(index=coins, inplace=True)
    return df.transpose()

############ Function Call ############
coin_info()

TypeError: unhashable type: 'dict'

## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [21]:
def hash_history():
  path = "data/blockchain-dot-com-hash-rate.json"
  hashes = pd.read_csv(path)
  return hashes

############ Function Call ############
hash_history()

,Timestamp,hash-rate
0,2020-07-15 00:00:00,1.189935e+08
1,2020-07-16 00:00:00,1.172690e+08
2,2020-07-17 00:00:00,1.017481e+08
3,2020-07-18 00:00:00,1.120953e+08
4,2020-07-19 00:00:00,1.129576e+08
...,...,...
360,2021-07-10 00:00:00,1.149524e+08
361,2021-07-11 00:00:00,8.924873e+07
362,2021-07-12 00:00:00,8.996272e+07
363,2021-07-13 00:00:00,9.710262e+07


# Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. In the html web scrape, some GPU names also had an asterisk associated with them. This made the data difficult to iterate through as the information I required did not have an even index pattern. I originally manually pulled out the names, until I realized the (*) was the issue and I removed them all at once. I could then find a pattern for indices and cut my code from 15 lines to 1.
   
2. The coins json in web collection 2 had a nested dictionary that had to be flattened to the top to make one two-way table. Without this fix there was just one column of `'coins'` with json dictionary text inside

3. The Global Power Plant Database had power generation data columns for a range of years as well as estimated genration columns for a smaller range of years. Different rows either had generation or estimated generation data, sometimes both, with nans in between. I created new average columns for both groups of generation data, and chose the larger or non empty one for each row to narrow it down to one average generation over a period of time per row.

4. The power sources listed in the power plant generation database and the power plant emissions database were not the same. I renamed a source in the emissions table to match the same source in the generation table. I collapsed some power generation sources into an 'other' category that the emissions table had. I then filled in 0s in the combined table for any renewable sources that do not emit pollutants.